<a href="https://colab.research.google.com/github/GalindoD/Backtesting-v1/blob/main/Backtester_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install backtrader yfinance matplotlib

In [2]:

import pandas as pd
import numpy as np
import plotly.express as px
import backtrader as bt
import yfinance as yf
import matplotlib.pyplot as plt
url = 'https://anaconda.org/conda-forge/libta-lib/0.4.0/download/linux-64/libta-lib-0.4.0-h166bdaf_1.tar.bz2'
!curl -L $url | tar xj -C /usr/lib/x86_64-linux-gnu/ lib --strip-components=1
!pip install conda-package-handling
!wget https://anaconda.org/conda-forge/ta-lib/0.5.1/download/linux-64/ta-lib-0.5.1-py311h9ecbd09_0.conda
!cph x ta-lib-0.5.1-py311h9ecbd09_0.conda
!mv ./ta-lib-0.5.1-py311h9ecbd09_0/lib/python3.11/site-packages/talib /usr/local/lib/python3.11/dist-packages/
import talib

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4039    0  4039    0     0   7307      0 --:--:-- --:--:-- --:--:--  7317
100  517k  100  517k    0     0   404k      0  0:00:01  0:00:01 --:--:-- 1386k
--2025-04-16 22:09:44--  https://anaconda.org/conda-forge/ta-lib/0.5.1/download/linux-64/ta-lib-0.5.1-py311h9ecbd09_0.conda
Resolving anaconda.org (anaconda.org)... 104.19.144.37, 104.19.145.37, 2606:4700::6813:9125, ...
Connecting to anaconda.org (anaconda.org)|104.19.144.37|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://binstar-cio-packages-prod.s3.amazonaws.com/5f738b4b78d4370a69f82984/673b62920c609ff6945bb24d?response-content-disposition=attachment%3B%20filename%3D%22ta-lib-0.5.1-py311h9ecbd09_0.conda%22%3B%20filename%2A%3DUTF-8%27%27ta-lib-0.5.1-py311h9ecbd09_0.conda&response-content-type=application%2Foctet-stream&X-Amz-Algorithm=

#Function for Data gathering

In [5]:
def gatherdata(ticker):
  data = yf.download(
      tickers = ticker,
      period="max",
      interval="5d",
      ignore_tz=True,
      auto_adjust=True)
  return data

def createdf(data, fastma, slowma, fastmacd, slowmacd, signal):
  data_df = data.copy()
  data_df.columns = ["close", "high", "low", "open", "volume"]
  data_df.drop(columns=["volume", "high", "low", "open"], inplace=True)

  data_df["R"] = data_df.close.pct_change().fillna(0)

  #Calculating values of Indicators
  data_df["slow_ma"] = talib.SMA(data_df.close, slowma)
  data_df["fast_ma"] = talib.SMA(data_df.close, fastma)
  data_df["macd"], data_df["signal"], data_df["histogram"] = talib.MACD(data_df.close, fastperiod=fastmacd, slowperiod=slowmacd, signalperiod=signal)

  #Delete First NA rows, resultring form lack of previous data
  data_df.dropna(inplace=True)
  return data_df

#Function for Computing Buy Signals

In [6]:

def signalcompute(data_df):
  data_df = data_df.assign(
      macdbuy = lambda x: np.where( x.macd > x.signal ,1,0)
      )
  data_df = data_df.assign(
      mabuy = lambda x: np.where( x.close > x.fast_ma ,1,0)
      )
  return data_df

In [7]:
def compute_strategy(data_df, strategy):
  data_df["buy"] = 0
  match strategy:
    case 1:
      #Strategy 1: MACD and MA, buy both signals 1 and sell both signals 0
      for i in range(len(data_df)):
        if ( data_df["macdbuy"].iloc[i] == 1 and data_df["mabuy"].iloc[i]  == 1):
          data_df["buy"].iloc[i] = 1
        elif ((data_df["macdbuy"].iloc[i] == 1 or data_df["mabuy"].iloc[i] == 1) and data_df["buy"].iloc[i-1] == 1):
          data_df["buy"].iloc[i] = 1
        else:
          data_df["buy"].iloc[i] = 0
    case 2:
      #Strategy 2: MACD and MA, buy both signals 1 and sell when either signal is 0
      for i in range(len(data_df)):
        if ( data_df["macdbuy"].iloc[i] == 1 and data_df["mabuy"].iloc[i] == 1):
            data_df["buy"].iloc[i] = 1
        else:
            data_df["buy"].iloc[i] = 0
    case 3:
      #Strategy 3: Only MACD
      for i in range(len(data_df)):
        if ( data_df["macdbuy"].iloc[i] == 1 ):
            data_df["buy"].iloc[i] = 1
        else:
            data_df["buy"].iloc[i] = 0

  #Shift
  data_df["buy"] = data_df["buy"].shift(1, fill_value=0)

  #Buying according to strategy
  data_df["R_strategy"] = data_df.R * data_df.buy

  #Assuming 5% return on liquid
  data_df['R_strategy'] = data_df['R_strategy'].replace([0], 0.0001369)

  #Calculating total Return
  returns = 100 * (1+data_df[["R","R_strategy"]]).prod()-1

  #Adding transaction costs to Strategy
  for i in range(len(data_df["buy"])):
      try:
        if data_df["buy"].iloc[i] != data_df["buy"].iloc[i+1]:
          data_df["R_strategy"].iloc[i+1] = data_df["R_strategy"].iloc[i+1] - trsncost
        else:
          continue
      except:
        print("End")


  return data_df



In [8]:
#Main Cell


#Variables
ticker = "BTC-USD"
#ticker = "QQQ"
#ticker = "TQQQ"

trsncost = 0.002
strategy = 3
slowma = 30
fastma = 10

fastmacd = 8
slowmacd = 17
signal = 9



#Gather the data
data = gatherdata(ticker)

#Create dataframe
data_df = createdf(data, fastma, slowma, fastmacd, slowmacd, signal)

#Compute the signal
data_df = signalcompute(data_df)

#Choose and compute strategy
data_df = compute_strategy(data_df, strategy)




Streaming output truncated to the last 5000 lines.
See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  data_df["buy"].iloc[i] = 1
<ipython-input-7-78befd71432a>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df["buy"].iloc[i] = 1
<ipython-input-7-78befd71432a>:24: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values

End


<ipython-input-7-78befd71432a>:24: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  data_df["buy"].iloc[i] = 1
<ipython-input-7-78befd71432a>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See th

In [9]:
#Total Return with transaction costs
returns = 100 * (1+data_df[["R","R_strategy"]]).prod()-1
print("With transaction cost:")
print("Return of Holding: ", returns.iloc[0])
print("Return of Strategy: ", returns.iloc[1])
print("The return of this strategy is ",round((returns.iloc[1]/returns.iloc[0]).item(), 4), "x of holding. Ticker: ", ticker)

With transaction cost:
Return of Holding:  37227.01822393945
Return of Strategy:  27028.01496584854
The return of this strategy is  0.726 x of holding. Ticker:  BTC-USD


In [10]:
px.line(100 * (1 + data_df[["R","R_strategy"]]).cumprod(), title="Total Return")

In [100]:
#Loop Cell


#Variables

ticker = "BTC-USD"
#ticker = "QQQ"
#ticker = "TQQQ"

trsncost = 0.002
strategy = 3
slowma = 30
fastma = 10

#Gather the data
data = gatherdata(ticker)

slowmacd = 17
signal = 5

returns_df = pd.DataFrame({'Values': ['MACD','Hold', 'Strategy', 'VS']})


for fastmacd in range(12, 18):
  #Create dataframe
  data_df = createdf(data, fastma, slowma, fastmacd, slowmacd, signal)

  #Compute the signal
  data_df = signalcompute(data_df)

  #Choose and compute strategy
  data_df = compute_strategy(data_df, strategy)

  #Total Return with transaction costs
  returns = 100 * (1+data_df[["R","R_strategy"]]).prod()-1

  column_name = "macd_" + str(fastmacd + 1)
  for row in range(3):
    list1 = [fastmacd, returns.iloc[0], returns.iloc[1], round((returns.iloc[1]/returns.iloc[0]).item(), 4)]
    returns_df.loc[row, column_name] = list1[row]

Streaming output truncated to the last 5000 lines.
<ipython-input-7-78befd71432a>:24: FutureWarning:

ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


<ipython-input-7-78befd71432a>:24: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice

End
End


Streaming output truncated to the last 5000 lines.
<ipython-input-7-78befd71432a>:26: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-7-78befd71432a>:26: FutureWarning:

ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating th

End
End


Streaming output truncated to the last 5000 lines.
<ipython-input-7-78befd71432a>:26: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-7-78befd71432a>:26: FutureWarning:

ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating th

End
End


<ipython-input-7-78befd71432a>:26: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-7-78befd71432a>:26: FutureWarning:

ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentat

In [101]:
returns_df

,Values,macd_13,macd_14,macd_15,macd_16,macd_17,macd_18
0,MACD,12.000000,13.000000,14.000000,15.000000,16.000000,17.000000
1,Hold,37153.650227,37153.650227,37153.650227,37153.650227,37153.650227,37153.650227
2,Strategy,23245.060272,29166.199877,27227.187517,28268.955994,27646.585930,109.721364
3,VS,NaN,NaN,NaN,NaN,NaN,NaN


In [94]:
from random import randint

returns_df = pd.DataFrame({'MACD': ['Hold', 'Strategy', 'VS']})

for column in range(12, 18):
    column_name = "macd_" + str(column + 1)
    for row in range(3):
      list1 = [1,2,3]
      #value = randint(0,10)
      #mean = np.mean(value)
      returns_df.loc[row, column_name] = list1[row]

In [95]:
returns_df

,MACD,macd_13,macd_14,macd_15,macd_16,macd_17,macd_18
0,Hold,1.0,1.0,1.0,1.0,1.0,1.0
1,Strategy,2.0,2.0,2.0,2.0,2.0,2.0
2,VS,3.0,3.0,3.0,3.0,3.0,3.0


In [43]:
data1 = [12, 200, 300,0.6]
data2 = [13, 300, 300,0.7]
data3 = [14, 400, 300,0.8]

In [44]:
df = df.assign(f=pd.Series(data2).values)

In [45]:
df

,Indicator,f
0,MACD,13.0
1,Hold,300.0
2,Strategy,300.0
3,VS,0.7


In [ ]:
data_df.to_excel('data.xlsx', index=True)

#Plot last 200 values

In [ ]:
last_200 = data_df.tail(200)


# Prepare plot
fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True)

#Plot SMA and Price
ax1.set_ylabel('Price', size=12)
last_200.close.plot(ax=ax1, color='r', label='Close')
last_200.slow_ma.plot(ax=ax1, color='b', label='SlowMA')
last_200.fast_ma.plot(ax=ax1, color='g', label='FastMA')
handles, labels = ax1.get_legend_handles_labels()
ax1.legend(handles, labels)

# Plot MACD
ax2.set_ylabel('MACD', size=12)
last_200.macd.plot(ax=ax2, color='b', label='Macd')
last_200.signal.plot(ax=ax2, color='g', label='Signal')
ax2.axhline(0, lw=2, color='0')
handles, labels = ax2.get_legend_handles_labels()
ax2.legend(handles, labels)


plt.show()

#Test code

In [ ]:
test_df = data_df.tail(50)

In [ ]:
test_df = test_df.drop(columns=['slow_ma','fast_ma', 'macd', 'signal', 'histogram'])

In [ ]:
test_df.drop(test_df.index[:5], inplace=True)

In [ ]:
test_df.drop(index=test_df.index[-10:], axis=0, inplace=True)

In [ ]:
test_df

In [ ]:
test2_df = test_df.copy()

In [ ]:
test2_df["buy"] = 0

In [ ]:
test2_df = test2_df.drop(columns=['close','R'])

In [ ]:
test2_df

In [ ]:
for i in range(len(test2_df)):
  if ( test2_df["macdbuy"].iloc[i] == 1 and test2_df["mabuy"].iloc[i] ) == 1:
#      print("Double 1")
#      print(test2_df["buy"].iloc[i].index())
      test2_df["buy"].iloc[i] = 1
#      test2_df.loc[i, "buy"] = 1
  elif ((test2_df["macdbuy"].iloc[i] == 1 or test2_df["mabuy"].iloc[i] == 1) and test2_df["buy"].iloc[i-1] == 1):
#      print("Single 1")
      test2_df["buy"].iloc[i] = 1
#      test2_df.loc[i, "buy"] = 1
  else:
#      print("NO BUY")
      test2_df["buy"].iloc[i] = 0
#      test2_df.loc[i, "buy"] = 0

In [ ]:
test2_df

In [ ]:
for i in range(len(test2_df["buy"])):
    try:
      if test2_df["buy"].iloc[i] != test2_df["buy"].iloc[i+1]:
        print("Different")
        test2_df["R_strategy"].iloc[i+1] = test2_df["R_strategy"].iloc[i+1] - trsncost
      else:
        continue
    except:
      print("End")

In [ ]:
test2_df